In [52]:
import numpy as np
import pandas as pd
import plotly_express as px
import plotly.graph_objs as go

In [2]:
from google.cloud import bigquery
client = bigquery.Client.from_service_account_json(
        'google_cloud_credentials.json')

In [3]:
def get_result_dataframe(query, index_col_name=None, datetime_cols=None, year_month_cols=None):
    query_job = client.query(query) 
    rows = []
    for row in query_job: 
        rows.append(pd.Series(dict(row.items())))
    df = pd.DataFrame(rows)
    if datetime_cols is not None:
        for datetime_col in datetime_cols:
            df[datetime_col] = pd.to_datetime(df[datetime_col], format='%Y%m%d', errors='ignore')
    if year_month_cols is not None:
        for year_month_col in year_month_cols:
            to_date = lambda ym: pd.Timestamp(year=int(ym / 100), month=int(ym % 100), day=1)
            df[year_month_col] = df[year_month_col].apply(to_date)
    if index_col_name is not None:
        df.index = df[index_col_name]
        df.drop([index_col_name], inplace=True, axis=1)
    return(df)

In [6]:
query = """
SELECT Actor1Geo_CountryCode AS Actor1Geo,
       Actor2Geo_CountryCode AS Actor2Geo,
       SUM(AvgTone * NumMentions) / SUM(NumMentions) * count(*) AS AvgTone
FROM `gdelt-bq.full.events`
WHERE MonthYear >= 201303
AND MonthYear < 201905
AND Actor1Geo_CountryCode IS NOT NULL
AND Actor2Geo_CountryCode IS NOT NULL
GROUP BY Actor1Geo_CountryCode, Actor2Geo_CountryCode
ORDER BY Actor1Geo_CountryCode, Actor2Geo_CountryCode
"""

In [7]:
df = get_result_dataframe(query)

In [8]:
#df.to_pickle('df_dump3.pickle')
df = pd.read_pickle('df_dump2.pickle')

In [108]:
treshold = 25000

actor1_geo_count = pd.read_csv('actor1_geo_count.csv').dropna()
actor2_geo_count = pd.read_csv('actor2_geo_count.csv').dropna()
a1_countries = set(actor1_geo_count[actor1_geo_count['Count'] >= treshold]['Actor1Geo_CountryCode'])
a2_countries = set(actor2_geo_count[actor2_geo_count['Count'] >= treshold]['Actor2Geo_CountryCode'])
countries_to_leave = list(a1_countries.intersection(a2_countries))
print(len(countries_to_leave), df.Actor1Geo.nunique(), df.Actor2Geo.nunique())

df = df[(df['Actor1Geo'].isin(countries_to_leave)) & (df['Actor2Geo'].isin(countries_to_leave))]
print(df.Actor1Geo.nunique(), df.Actor2Geo.nunique())

(210, 252, 252)
(210, 210)


In [109]:
df.shape[0], len(countries_to_leave) ** 2

(44100, 44100)

In [110]:
df.sort_values(['Actor1Geo', 'Actor2Geo'], inplace=True)

In [111]:
from itertools import product

index = pd.MultiIndex.from_product((countries_to_leave, countries_to_leave),
                                   names=['Actor1Geo', 'Actor2Geo']
                                  )

df.index = pd.MultiIndex.from_arrays((df['Actor1Geo'], df['Actor2Geo']),
                                     names=['Actor1Geo', 'Actor2Geo']
                                    )
df.drop(['Actor1Geo', 'Actor2Geo'], axis=1, inplace=True)

df = df.reindex(index, fill_value=0.0).reset_index()


assert df.groupby('Actor1Geo')['Actor2Geo'].count().nunique() == 1
df.sort_values(['Actor1Geo', 'Actor2Geo'], inplace=True)

In [112]:
dist_matrix = df['AvgTone'].values.reshape((-1, df.groupby('Actor1Geo')['Actor2Geo'].count().unique()[0]))

In [113]:
labels_idx = pd.Series(df['Actor1Geo'].unique(), name='country_id')
country_index_to_name_mapping = pd.read_csv('FIPS.country.csv', 
                                            sep='\t',
                                            names=['index', 'name'],
                                            index_col='index'
                                           )['name']
labels = labels_idx.map(country_index_to_name_mapping)

In [115]:
from sklearn.manifold import TSNE
model = TSNE(
    perplexity=5,
    early_exaggeration=10,
    learning_rate=50,
)

In [98]:
from sklearn.manifold import MDS
model = MDS(dissimilarity='precomputed')

In [116]:
transformed = model.fit_transform((dist_matrix + dist_matrix.T) / 2)

transformed_df = pd.DataFrame({"x": transformed[:,0],
                               "y": transformed[:,1],
                               "label": labels,
                              })

In [117]:
px.scatter(transformed_df,
           x="x",
           y="y",
           hover_name='label'
          )

In [32]:
pd.options.display.max_rows = 150

In [239]:
from sklearn.cluster import AffinityPropagation

model = AffinityPropagation(affinity='precomputed')

clusters = model.fit_predict(dist_matrix)

In [263]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(
    n_clusters=20,
#     affinity='precomputed',
    linkage='ward',
    compute_full_tree=True,
)

clusters = model.fit_predict(dist_matrix)

In [252]:
from sklearn.cluster import MeanShift

model = MeanShift()
clusters = model.fit_predict(dist_matrix.T)

In [236]:
from sklearn.cluster import Birch

model = Birch(
    branching_factor=10,
    n_clusters=20
)

clusters = model.fit_predict(dist_matrix.T)

In [229]:
from sklearn.cluster import KMeans

model = KMeans(
    n_clusters=25
)
clusters = model.fit_predict(dist_matrix.T)

In [211]:
np.unique((dist_matrix + dist_matrix.T) / 2)

array([-16.15079005, -13.23985419, -12.99243783, ...,  10.26785714,
        11.11111111,  11.9900239 ])

In [214]:
from sklearn.cluster import SpectralClustering

model = SpectralClustering(
    n_clusters=20,
    affinity='nearest_neighbors',
)

clusters = model.fit_predict(dist_matrix)

In [ ]:
pd.Series(clusters, index=labels).sort_values()

In [264]:
cluster_df = pd.concat([pd.Series(clusters, name='cluster_id'), labels_idx], axis=1)
fips_to_iso_mapping = pd.read_csv('fips-iso-country.csv', index_col='FIPS')
cluster_df = cluster_df.join(fips_to_iso_mapping,
                             on=['country_id'],
                             how='right',
                            ).fillna(-1)
cluster_df.rename({'ISO': 'country_iso'}, axis=1, inplace=True)

In [265]:
fig = px.choropleth(
    cluster_df,
    locations='country_iso',
    locationmode='ISO-3',
    color='cluster_id',
    labels='cluster_id',
    color_continuous_scale=px.colors.qualitative.Pastel,
)
fig.show()

In [108]:
query = """
SELECT Actor1Geo_CountryCode AS ActorGeo,
       Actor1Type1Code AS Type1,
       Actor2Type1Code AS Type2,
       SUM(AvgTone * NumMentions) / SUM(NumMentions) * count(*) AS AvgTone
FROM `gdelt-bq.full.events`
WHERE MonthYear >= 201303
AND MonthYear < 201905
AND Actor1Geo_CountryCode IS NOT NULL
AND Actor1Geo_CountryCode = Actor2Geo_CountryCode 
GROUP BY Actor1Geo_CountryCode, Actor1Type1Code, Actor2Type1Code
ORDER BY Actor1Geo_CountryCode, Actor1Type1Code, Actor2Type1Code
"""

In [109]:
df = get_result_dataframe(query)
df.to_pickle('df_dump4.pickle')

In [238]:
df = pd.read_pickle('df_dump4.pickle')

In [239]:
threshold = 50000

actor1_geo_count = pd.read_csv('actor1_geo_count.csv').dropna()
actor2_geo_count = pd.read_csv('actor2_geo_count.csv').dropna()
a1_countries = set(actor1_geo_count[actor1_geo_count['Count'] >= threshold]['Actor1Geo_CountryCode'])
a2_countries = set(actor2_geo_count[actor2_geo_count['Count'] >= threshold]['Actor2Geo_CountryCode'])
countries_to_leave = list(a1_countries.intersection(a2_countries))

In [240]:
df = df[(df['ActorGeo'].isin(countries_to_leave))]

In [241]:
multi_index = pd.MultiIndex.from_product(
    [df['ActorGeo'].unique(), df['Type1'].unique(), df['Type2'].unique()],
    names=['ActorGeo', 'Type1', 'Type2'])
df.index = pd.MultiIndex.from_arrays(
    [df['ActorGeo'], df['Type1'], df['Type2']],
    names=['ActorGeo', 'Type1', 'Type2'])
df.drop(['ActorGeo', 'Type1', 'Type2'], axis=1, inplace=True)
df = df.reindex(multi_index).reset_index()
df.fillna(0., inplace=True)
df.sort_values(['ActorGeo', 'Type1', 'Type2'], inplace=True)
df.index = np.arange(df.shape[0])

In [198]:
df1 = df.groupby(['ActorGeo', 'Type1']).sum().reset_index()
df1.rename({'Type1': 'Type'}, axis=1, inplace=True)
df1['Type'] = df1['Type'] + '_1'
df2 = df.groupby(['ActorGeo', 'Type2']).sum().reset_index()
df2.rename({'Type2': 'Type'}, axis=1, inplace=True)
df2['Type'] = df2['Type'] + '_2'
df = pd.concat([df1, df2])
df.sort_values(['ActorGeo', 'Type'], inplace=True)

In [244]:
df = df[(df.Type1 != 0.0) & df.Type2 != 0.0]

In [246]:
types_no = np.unique(df.groupby('ActorGeo')['AvgTone'].count())[0]

In [247]:
data = df['AvgTone'].values.reshape((-1, types_no))
labels = df['ActorGeo'].unique()

In [248]:
norm_data = (data - np.mean(data, axis=1)[:, np.newaxis]) / np.std(data, axis=1)[:, np.newaxis]

In [203]:
fips_to_iso_mapping = pd.read_csv('fips-iso-country.csv', index_col='FIPS')

In [249]:
from sklearn.cluster import AffinityPropagation

model = AffinityPropagation()

clusters = model.fit_predict(norm_data)

In [230]:
from sklearn.cluster import Birch

model = Birch(
    branching_factor=5,
    n_clusters=25
)

clusters = model.fit_predict(norm_data)

In [234]:
from sklearn.cluster import KMeans

model = KMeans(
    n_clusters=25
)
clusters = model.fit_predict(norm_data)

In [253]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(
    n_clusters=30,
    linkage='ward',
)
clusters = model.fit_predict(norm_data)

In [254]:
cluster_df = pd.DataFrame({
    'country_id': labels,
    'cluster_id': clusters,
})

In [255]:
cluster_df = cluster_df.join(fips_to_iso_mapping,
                             on=['country_id'],
                             how='right',
                            ).fillna(-1)
cluster_df.rename({'ISO': 'country_iso'}, axis=1, inplace=True)

In [256]:
px.choropleth(
    cluster_df,
    locations='country_iso',
    locationmode='ISO-3',
    color='cluster_id',
    labels='cluster_id',
    color_continuous_scale=px.colors.qualitative.Alphabet,
    projection='natural earth',
)

In [257]:
from sklearn.manifold import TSNE
model = TSNE(
    perplexity=5,
    early_exaggeration=10,
    learning_rate=50,
)

In [258]:
transformed = model.fit_transform(norm_data)

transformed_df = pd.DataFrame({"x": transformed[:,0],
                               "y": transformed[:,1],
                               "label": labels,
                              })
px.scatter(transformed_df,
           x="x",
           y="y",
           hover_name='label'
          )